Start with the same imports as before

In [ ]:
import os
os.chdir('/data/kkovacs/Python')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import scanpy as sc
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import gdown
import sankey
import anndata
import pandas as pd
from pySankey.sankey import sankey
from matplotlib.pyplot import rc_context
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(5, 5))
torch.set_printoptions(precision=3, sci_mode=False, edgeitems=12)

Reload all healthy sample objects, those with the 2000 gene subsetted

In [2]:
directory = '/data/kkovacs/Python'

In [3]:
covid = sc.read("/data/kkovacs/Python/Data/Mouse_Covid_2k_h.h5ad")
copd_covid = sc.read(directory + "/Data/Mouse_Copd_Covid_2k_h.h5ad")
copd = sc.read(directory + "/Data/Mouse_Copd_2k_h.h5ad")
age = sc.read(directory + "/Data/Mouse_Age_2k_h.h5ad")
cssc = sc.read(directory + "/Data/Mouse_COPD_SHAM_SARSCov2_2k_h.h5ad")
herpes = sc.read(directory + "/Data/Mouse_Herpesvirus_2k_h.h5ad")
hyper = sc.read(directory + "/Data/Mouse_Hyperoxia_2k_h.h5ad")
tb = sc.read(directory + "/Data/Mouse_TB_2k_h.h5ad")
cancer = sc.read(directory + "/Data/Mouse_Cancer_2k_h.h5ad")
hypo = sc.read(directory + "/Data/Mouse_Hypoxia_2k_h.h5ad")
rad = sc.read(directory + "/Data/Mouse_Radiation_2k_h.h5ad")
sendai = sc.read(directory + "/Data/Mouse_Post_Sendai_2k_h.h5ad")
tumor = sc.read(directory + "/Data/Mouse_TumorKO_2k_h.h5ad")
fibro = sc.read(directory + "/Data/Mouse_fibrosis_2k_h.h5ad")
asthma = sc.read(directory + "/Data/Mouse_asthma_2k_h.h5ad")
fibage = sc.read(directory + "/Data/Mouse_fibro_age_2k_h.h5ad")
influ = sc.read(directory + "/Data/Mouse_influenza_2k_h.h5ad")
print(covid)

AnnData object with n_obs × n_vars = 28427 × 2000
    obs: 'nCount_covid', 'nFeature_covid', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'CellStatus', 'pANN_0.25_0.09_241', 'DF.classifications_0.25_0.09_241', 'DoubletStatus', 'pANN_0.25_0.09_520', 'DF.classifications_0.25_0.09_520', 'pANN_0.25_0.09_148', 'DF.classifications_0.25_0.09_148', 'pANN_0.25_0.09_368', 'DF.classifications_0.25_0.09_368', 'pANN_0.25_0.09_1110', 'DF.classifications_0.25_0.09_1110', 'pANN_0.25_0.09_33', 'DF.classifications_0.25_0.09_33', 'pANN_0.25_0.09_1015', 'DF.classifications_0.25_0.09_1015', 'nGene', 'nUMI', 'Group', 'res.0', 'res.0.1', 'res.0.2', 'res.0.3', 'res.0.4', 'res.0.5', 'res.0.6', 'res.0.7', 'res.0.8', 'res.0.9', 'res.1', 'seurat_clusters', 'infection', 'Sample.Gender', 'percent.hb', 'percent.rp', 'cluster_labels_res.0.4', 'predicted.classification_Angelidis.score', 'predicted.classification_Angelidis', 'mouseRNA.labels', 'immgen.labels', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn

Create a list of all the adata objects

In [4]:
sets = [covid,copd_covid,copd,age,cssc,herpes,hyper,tb,cancer,hypo,rad,\
	sendai,tumor,fibro,asthma,fibage,influ]

def check_non_integer_values(adatas):
    # Loop through each adata object in the list
    for idx, adata in enumerate(adatas):
        # Assuming adata is an AnnData object
        if not isinstance(adata, anndata.AnnData):
            raise ValueError(f"Input at index {idx} is not an AnnData object.")
        study = str(adata.obs['study'].unique()[0])
        non_integer_values = np.any(adata.X.data % 1 != 0)
        # Print information about non-integer values
        if non_integer_values:
            print(f"Study: {study} - Non-integer values found.")
            # Add additional information or actions as needed
        else:
            print(f"Study: {study} - All values are integers.")

check_non_integer_values(sets)

Study: Mouse_Covid - All values are integers.
Study: Mouse_Copd_Covid - All values are integers.
Study: Mouse_Copd - All values are integers.
Study: Mouse_Age - All values are integers.
Study: Mouse_COPD_SHAM_SARSCov2 - All values are integers.
Study: Mouse_Herpesvirus - All values are integers.
Study: Mouse_Hyperoxia - All values are integers.
Study: Mouse_TB - All values are integers.
Study: Mouse_Cancer - All values are integers.
Study: Mouse_Hypoxia - All values are integers.
Study: Mouse_Radiation - All values are integers.
Study: Mouse_Post_Sendai - All values are integers.
Study: Mouse_TumorKO - All values are integers.
Study: Mouse_fibrosis - All values are integers.
Study: Mouse_asthma - All values are integers.
Study: Mouse_fibro_age - All values are integers.
Study: Mouse_influenza - All values are integers.


In [5]:
adata = [covid,copd_covid,copd,age,cssc,herpes,hyper,\
	tb,cancer,hypo,rad,sendai,tumor,fibro,asthma,fibage,influ]
stages = [covid]
FL = []

Using the loop below we identify a condition key and celltype key for each ref and query. These help in the integratoin methods, in which will help split the neighbours up via the cell type key, but also merge as best as possible on the condition key, being the different data studies.

In [ ]:
for i in range(0,len(adata)-1):
	source_adata = stages[i]
	target_adata = adata[i+1]
	source_condition_key = 'study'
	source_cell_type_key = 'cell_type'
	target_condition_key = 'study'
	target_cell_type_key = 'CellTypes'
	source_adata =source_adata.copy()
	sca.models.SCVI.setup_anndata(source_adata, batch_key=source_condition_key, labels_key=source_cell_type_key)
	vae = sca.models.SCVI(
		source_adata,
		n_layers=2,
		encode_covariates=True,
		deeply_inject_covariates=False,
		use_layer_norm="both",
		use_batch_norm="none")
	vae.train(max_epochs=20)
	scanvae = sca.models.SCANVI.from_scvi_model(vae, unlabeled_category = "Unknown")
	scanvae.train(max_epochs=20)
	reference_latent = sc.AnnData(scanvae.get_latent_representation())
	reference_latent.obs["cell_type"] = source_adata.obs[source_cell_type_key].tolist()
	reference_latent.obs["batch"] = source_adata.obs[source_condition_key].tolist()
	sc.pp.neighbors(reference_latent)
	sc.tl.leiden(reference_latent)
	sc.tl.umap(reference_latent)
	reference_latent.obs['predictions'] = scanvae.predict()
	ref_path = 'ref_model_healthy/'
	scanvae.save(ref_path, overwrite=True)

Using SCANVI to create a model based on the reference object, 'covid' we are able to map this model onto our query, which will then be repeated once the both ref and query are concatenated together and looped back through.

In [ ]:
	model = sca.models.SCANVI.load_query_data(
		target_adata,
		ref_path,
		freeze_dropout = True)	
	model.train(
		max_epochs=20,
		plan_kwargs=dict(weight_decay=0.0),
		check_val_every_n_epoch=10)
	query_latent = sc.AnnData(model.get_latent_representation())
	query_latent.obs['cell_type'] = target_adata.obs["CellTypes"].tolist()
	query_latent.obs['batch'] = target_adata.obs[target_condition_key].tolist()
	sc.pp.neighbors(query_latent)
	sc.tl.leiden(query_latent)
	sc.tl.umap(query_latent)
	surgery_path = 'surgery_model_healthy'
	model.save(surgery_path, overwrite=True)
	query_latent.obs['predictions'] = model.predict()

Once the query has been made of the additional studies, we move onto the combining of the reference and query to make a model of the conjoined studies

In [ ]:
    adata_full = source_adata.concatenate(target_adata,index_unique=None,join='outer',fill_value=0)
	full_latent = sc.AnnData(model.get_latent_representation(adata=adata_full))
	full_latent.obs['cell_type'] = adata_full.obs[source_cell_type_key].tolist()
	full_latent.obs['batch'] = adata_full.obs[source_condition_key].tolist()
	sc.pp.neighbors(full_latent)
	sc.tl.leiden(full_latent)
	sc.tl.umap(full_latent)
	full_latent.obs['predictions'] = model.predict(adata=adata_full)
	sc.pp.neighbors(full_latent)
	sc.tl.leiden(full_latent)
	sc.tl.umap(full_latent)
	full_latent.obs['study'] = adata_full.obs['study'].tolist()
	adata_full.obsm['X_umap'] = full_latent.obsm['X_umap']
	sc.pl.umap(
		full_latent,
		color=["predictions", "study"],
		frameon=False,
		wspace=0.6,
		show = None,
		save = "healthy_comp_{}.png".format(i)
	)
	FL.insert(i,full_latent)
	adata_full.obs['predictions']=full_latent.obs['predictions'].tolist()
	adata_full.obs['cell_type']=full_latent.obs['cell_type'].tolist()
	for col in adata_full.obs.columns:
		if pd.api.types.is_object_dtype(adata_full.obs[col]):
			adata_full.obs[col]=adata_full.obs[col].astype(str)
	stages.insert(i+1,adata_full)

In [ ]:
Full_adata = covid.concatenate(copd_covid,copd,age,cssc,herpes,hyper,\
	tb,cancer,hypo,rad,sendai,tumor,fibro,asthma,fibage,influ,\
	index_unique=None,join='outer',fill_value=0)

#creating all gene healthy, sending over all integration data over to all gene object
sc.tl.leiden(full_latent, resolution=1, key_added='leiden_1')
sc.tl.leiden(full_latent, resolution=2, key_added='leiden_2')
sc.tl.leiden(full_latent, resolution=3, key_added='leiden_3')
sc.tl.leiden(full_latent, resolution=4, key_added='leiden_4')

Full_adata.obsm['X_umap'] = full_latent.obsm['X_umap']

Full_adata.obsm['X_MLCA'] = full_latent.X
Full_adata.obs['leiden_1'] = full_latent.obs['leiden_1'].copy()
Full_adata.obs['leiden_2'] = full_latent.obs['leiden_2'].copy()
Full_adata.obs['leiden_3'] = full_latent.obs['leiden_3'].copy()


common_genes = set(Full_adata.var_names) & set(full_latent.var_names)

Full_adata.var['highly_variable'] = [gene in common_genes for gene in Full_adata.var_names]


for col in Full_adata.obs.columns:
	if pd.api.types.is_object_dtype(Full_adata.obs[col]):
		Full_adata.obs[col]=Full_adata.obs[col].astype(str)

Full_adata.__dict__['_raw'].__dict__['_var'] = Full_adata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})

Full_adata.write(directory + "/Data/Full_adata_h_pre.h5ad")

In [ ]:
This is then repeated for the disease sets also. The Healthy set is focused on post 
integration to ensure definiative celltypes for the diseased to be integrated with